# EDA - Bases de données patient, consommant et hospitalisation

### Installation libraire requise

In [ ]:
!pip install pandas_profiling

### Import librairies

In [1]:
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

### Import des bases de données

In [2]:
hospit=pd.read_csv("base_hospit.csv", sep=';')
patient=pd.read_csv("base_patient.csv", sep=';')
consommant=pd.read_csv("consommant.csv", sep=';')

### Function utile

In [3]:
def count():
    print(f"Nb patients = {len(patient)}")
    print(f"Nb hospit = {len(hospit)}")
    print(f"Nb conso = {len(consommant)}")

### Conversion des dates en datetime

In [4]:
hospit['EXE_SOI_DTD']=pd.to_datetime(hospit['EXE_SOI_DTD'], dayfirst=True)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].fillna(np.nan).astype(float)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].astype('Int64')

In [5]:
consommant['datemax']=pd.to_datetime(consommant['datemax'], dayfirst=True)

In [6]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)
patient['dte_deces']=pd.to_datetime(patient['dte_deces'], dayfirst=True)

## Encodage des patients

In [7]:
dict_patients = {}
for i in range(len(patient)):
    code = patient.CODE_PATIENT.iloc[i]
    dict_patients[code] = i

In [8]:
patient['CODE_PATIENT_ENC'] = ['P' + str(i) for i in range(len(patient))]

In [9]:
enc_conso = []
for i in range(len(consommant)):
    code = consommant.CODE_PATIENT.iloc[i]
    patient_n = dict_patients[code]
    enc_conso.append('P'+str(patient_n))
consommant['CODE_PATIENT_ENC'] = enc_conso

In [10]:
enc_hospit = []
for i in range(len(hospit)):
    code = hospit.BEN_NIR_IDT.iloc[i]
    patient_n = dict_patients[code]
    enc_hospit.append('P' + str(patient_n))
hospit['BEN_NIR_IDT_ENC'] = enc_hospit

In [11]:
patient=patient.drop(columns=['CODE_PATIENT'])
consommant=consommant.drop(columns=['CODE_PATIENT'])
hospit=hospit.drop(columns=['BEN_NIR_IDT'])

In [12]:
import pickle
    
with open('dict_codes_patients.pickle', 'wb') as handle:
    pickle.dump(dict_patients, handle)

In [13]:
patient=patient.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
consommant=consommant.rename(columns={'CODE_PATIENT_ENC':'CODE_PATIENT'})
hospit=hospit.rename(columns={'BEN_NIR_IDT_ENC':'BEN_NIR_IDT'})

## Suppression des patients dont la première hospitalisation est postérieure à 2016 ou antérieure à 2010

In [14]:
count()

Nb patients = 24311
Nb hospit = 226127
Nb conso = 24232


In [15]:
patient=patient[(patient['date_h0']<'01-01-2016')&(patient['date_h0']>='01-01-2010')]

In [16]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)

In [17]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [18]:
consommant.head()

_TEMG001    datemax CODE_PATIENT
0     21769 2019-08-08           P0
1     20150 2015-03-03           P6
2     20312 2015-08-12           P8
3     21545 2018-12-27          P10
4     21769 2019-08-08          P13

In [19]:
hospit.head()

RSA_NUM    ETA_NUM EXE_SOI_DTD  SEJ_NBJ GRG_GHM DGN_PAL DGN_REL BEN_NIR_IDT
0  144689  310781406  2011-11-22        2  05K051   I2100     NaN          P0
1    2763  110780061  2011-11-24        8  05M042  I21000     NaN          P0
2   15548  110780061  2011-12-08        0  05M16T    I255     NaN          P0
3    5260  110780061  2012-03-04        1  05M09T    I501     NaN          P0
4  112258  310781406  2012-03-05        7  05M092    I501     NaN          P0

In [20]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [21]:
hospit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158715 entries, 0 to 158714
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   RSA_NUM      158715 non-null  object        
 1   ETA_NUM      158715 non-null  object        
 2   EXE_SOI_DTD  158713 non-null  datetime64[ns]
 3   SEJ_NBJ      155779 non-null  Int64         
 4   GRG_GHM      158715 non-null  object        
 5   DGN_PAL      158715 non-null  object        
 6   DGN_REL      68448 non-null   object        
 7   BEN_NIR_IDT  158715 non-null  object        
dtypes: Int64(1), datetime64[ns](1), object(6)
memory usage: 9.8+ MB


In [22]:
count()

Nb patients = 13310
Nb hospit = 158715
Nb conso = 13241


## On conserve uniquement les mois de décès

In [23]:
patient['dte_deces']=patient['dte_deces'].dt.to_period('M')

In [24]:
patient.head()

date_h0  ALD_before  pop  y_nais BEN_RES_DPT  BEN_SEX_COD dte_deces  \
0  2011-11-22           0    1    1949          11            1       NaT   
6  2013-12-05           0    1    1929          14            2   2015-03   
8  2015-07-02           0    1    1967          54            2   2015-08   
10 2014-11-21           1    0    1932          30            1   2018-12   
13 2011-05-19           0    1    1959         974            1       NaT   

   CODE_PATIENT  
0            P0  
6            P6  
8            P8  
10          P10  
13          P13

In [25]:
count()

Nb patients = 13310
Nb hospit = 158715
Nb conso = 13241


## On conserve uniquement les patients majeurs au moment de leur première hospitalisation

#### Nombre de patients mineurs dans les données

In [26]:
np.unique(((pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais) < 18), return_counts=True)

(array([False,  True]), array([13220,    90]))

In [27]:
patient = patient[pd.to_datetime(patient.date_h0).apply(lambda x : x.year) - patient.y_nais >= 18]

In [28]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [29]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [30]:
patient

date_h0  ALD_before  pop  y_nais BEN_RES_DPT  BEN_SEX_COD dte_deces  \
0     2011-11-22           0    1    1949          11            1       NaT   
6     2013-12-05           0    1    1929          14            2   2015-03   
8     2015-07-02           0    1    1967          54            2   2015-08   
10    2014-11-21           1    0    1932          30            1   2018-12   
13    2011-05-19           0    1    1959         974            1       NaT   
...          ...         ...  ...     ...         ...          ...       ...   
24302 2014-02-08           0    1    1920          29            1   2014-12   
24303 2012-02-09           0    1    1927          61            1   2017-05   
24304 2010-07-27           0    1    1924         999            1   2010-08   
24308 2011-11-25           0    1    1925          13            2   2013-03   
24310 2010-12-18           0    1    1941          81            2       NaT   

      CODE_PATIENT  
0               P0  
6               P6  
8               P8  
10             P10  
13             P13  
...            ...  
24302       P24302  
24303       P24303  
24304       P24304  
24308       P24308  
24310       P24310  

[13220 rows x 8 columns]

In [31]:
count()

Nb patients = 13220
Nb hospit = 157199
Nb conso = 13151


## On retire les patients ayant déjà connu une hospitalisation avant l'étude (pop=0)

In [32]:
patient=patient[patient["pop"]==1].reset_index(drop=True)

In [33]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [34]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [35]:
patient = patient.drop(columns=['pop'])

In [36]:
count()

Nb patients = 10204
Nb hospit = 157199
Nb conso = 10138


## On retire les patients sans consommation depuis plus de 5 ans, non déclarés décédés

In [37]:
patient_non_décédés=patient[patient['dte_deces'].fillna(0)==0]["CODE_PATIENT"]

In [38]:
patients_sans_conso=consommant[(consommant["CODE_PATIENT"].isin(patient_non_décédés))&(consommant['datemax']<'01-01-2014')]['CODE_PATIENT']

In [39]:
patient = patient[~patient["CODE_PATIENT"].isin(patients_sans_conso)].reset_index(drop=True)

In [40]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [41]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [42]:
count()

Nb patients = 10052
Nb hospit = 156336
Nb conso = 9986


## Ajout des insuffisances cardiaques avec état de choc cardiogénique

In [43]:
#Insuffisances cardiaques et états de choc circulatoire
#hospit[hospit['GRG_GHM'].str.startswith('05M09')]

#Insuffisances cardiaques et états de choc cardiogénique (code R57)
#hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))]

#Ajout d'une colonne CHOC
hospit['CHOC']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))|(hospit['DGN_REL'].str.startswith('R57'))].index)
hospit['CHOC']=hospit['CHOC'].astype('int')

In [44]:
hospit.CHOC.sum()

478

## Ajout des hémodialyses

In [47]:
hospit['HEMOD']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z49'))].index)
hospit['HEMOD']=hospit['HEMOD'].astype('int')

In [48]:
hospit['DGN_PAL'].str.startswith('Z49').sum()

51150

## Ajout des chimiothérapies

In [49]:
hospit['CHIMIO']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z511'))|(hospit['DGN_PAL'].str.startswith('Z512'))].index)
hospit['CHIMIO']=hospit['CHIMIO'].astype('int')

In [50]:
hospit['DGN_PAL'].str.startswith('Z511').sum(), hospit['DGN_PAL'].str.startswith('Z512').sum()

(9819, 1822)

## Ajout des radiothérapies

In [51]:
hospit['Radiotherapie']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z510'))|(hospit['DGN_REL'].str.startswith('Z510'))].index)
hospit['Radiotherapie']=hospit['Radiotherapie'].astype('int')

In [52]:
hospit.Radiotherapie.sum()

5660

## Ajout des transfusions

In [53]:
hospit['Transfusion']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('28Z14Z'))].index)
hospit['Transfusion']=hospit['Transfusion'].astype('int')

In [54]:
hospit.Transfusion.sum()

2291

## Statistiques descriptives sur les datasets

### Hospitalisation

In [55]:
hospit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156336 entries, 0 to 156335
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   RSA_NUM        156336 non-null  object        
 1   ETA_NUM        156336 non-null  object        
 2   EXE_SOI_DTD    156334 non-null  datetime64[ns]
 3   SEJ_NBJ        153403 non-null  Int64         
 4   GRG_GHM        156336 non-null  object        
 5   DGN_PAL        156336 non-null  object        
 6   DGN_REL        67298 non-null   object        
 7   BEN_NIR_IDT    156336 non-null  object        
 8   CHOC           156336 non-null  int64         
 9   HEMOD          156336 non-null  int64         
 10  CHIMIO         156336 non-null  int64         
 11  Radiotherapie  156336 non-null  int64         
 12  Transfusion    156336 non-null  int64         
dtypes: Int64(1), datetime64[ns](1), int64(5), object(6)
memory usage: 15.7+ MB


In [52]:
profile_hospit = ProfileReport(hospit, title="Hospit Profile Report")

In [53]:
profile_hospit.to_file("profile_hospit.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Consommant

In [54]:
consommant.head()

_TEMG001    datemax CODE_PATIENT
0     21769 2019-08-08           P0
1     20150 2015-03-03           P6
2     20312 2015-08-12           P8
3     21769 2019-08-08          P13
4     21787 2019-08-26          P16

In [55]:
consommant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9986 entries, 0 to 9985
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _TEMG001      9986 non-null   int64         
 1   datemax       9986 non-null   datetime64[ns]
 2   CODE_PATIENT  9986 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 234.2+ KB


In [56]:
profile_conso = ProfileReport(consommant, title="Consommant Profile Report")

In [57]:
profile_conso.to_file("profile_conso.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Patient

In [58]:
patient.head()

date_h0  ALD_before  y_nais BEN_RES_DPT  BEN_SEX_COD dte_deces  \
0 2011-11-22           0    1949          11            1       NaT   
1 2013-12-05           0    1929          14            2   2015-03   
2 2015-07-02           0    1967          54            2   2015-08   
3 2011-05-19           0    1959         974            1       NaT   
4 2010-03-18           0    1947          76            1       NaT   

  CODE_PATIENT  
0           P0  
1           P6  
2           P8  
3          P13  
4          P16

In [59]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10052 entries, 0 to 10051
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date_h0       10052 non-null  datetime64[ns]
 1   ALD_before    10052 non-null  int64         
 2   y_nais        10052 non-null  int64         
 3   BEN_RES_DPT   10052 non-null  object        
 4   BEN_SEX_COD   10052 non-null  int64         
 5   dte_deces     6619 non-null   period[M]     
 6   CODE_PATIENT  10052 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2), period[M](1)
memory usage: 549.8+ KB


In [60]:
profile_patient = ProfileReport(patient, title="Patient Profile Report")

In [61]:
profile_patient.to_file("profile_patient.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

####

#### Nombres de patients et d'hospitalisations après processing

In [56]:
#Nombre de patients
patient_init = pd.read_csv("base_patient.csv", sep=';')
print(f"Il y avait {len(patient_init)} patients initialement. Suite à l'ensemble des critères de rejet, il en reste {len(patient)}.")
print(f"On en a enlevé {len(patient_init) - len(patient)}.")

Il y avait 24311 patients initialement. Suite à l'ensemble des critères de rejet, il en reste 10052.
On en a enlevé 14259.


In [57]:
hospit_init = pd.read_csv("base_hospit.csv", sep=';')
print(f"Il y avait {len(hospit_init)} hospitalisations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(hospit)}.")
print(f"On en a enlevé {len(hospit_init) - len(hospit)}.")

Il y avait 226127 hospitalisations initialement. Suite à l'ensemble des critères de rejet, il en reste 156336.
On en a enlevé 69791.


In [64]:
conso_init = pd.read_csv("consommant.csv", sep=';')
print(f"Il y avait {len(conso_init)} consommations initialement. Suite à l'ensemble des critères de rejet, il en reste {len(consommant)}.")
print(f"On en a enlevé {len(conso_init) - len(consommant)}.")

Il y avait 24232 consommations initialement. Suite à l'ensemble des critères de rejet, il en reste 9986.
On en a enlevé 14246.


#### Nombre de patients dialysés

###### Chimiothérapie

In [58]:
hospit_chimio = hospit.loc[hospit.CHIMIO == 1]
nb_patients_chimio = hospit_chimio.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_chimio)} hospitalisations pour des raisons de chimiothérapie.")
print(f"Cela concerne {nb_patients_chimio} patients.")

Il y a 11641 hospitalisations pour des raisons de chimiothérapie.
Cela concerne 916 patients.


###### Hémodialyse

In [59]:
hospit_hemod = hospit.loc[hospit.HEMOD == 1]
nb_patients_hemod = hospit_hemod.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_hemod)} hospitalisations pour des raisons d'hémodialyse.")
print(f"Cela concerne {nb_patients_hemod} patients.")

Il y a 51150 hospitalisations pour des raisons d'hémodialyse.
Cela concerne 343 patients.


###### Radiothérapie

In [60]:
hospit_radiotherapie = hospit.loc[hospit.Radiotherapie == 1]
nb_patients_radiotherapie = hospit_radiotherapie.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_radiotherapie)} hospitalisations pour des raisons de radiothérapie.")
print(f"Cela concerne {nb_patients_radiotherapie} patients.")

Il y a 5660 hospitalisations pour des raisons de radiothérapie.
Cela concerne 302 patients.


###### Transfusions

In [61]:
hospit_transfusion = hospit.loc[hospit.Transfusion == 1]
nb_patients_transfusion = hospit_transfusion.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_transfusion)} hospitalisations pour des raisons de transfusion.")
print(f"Cela concerne {nb_patients_transfusion} patients.")

Il y a 2291 hospitalisations pour des raisons de transfusion.
Cela concerne 382 patients.


###### Hospitalisations autres

In [62]:
hospit_non_chronique = hospit.loc[(hospit.HEMOD == 0) & (hospit.CHIMIO==0) & (hospit.Radiotherapie==0) & (hospit.Transfusion==0)]
nb_patients_non_chronique = hospit_non_chronique.BEN_NIR_IDT.nunique()
print(f"Il y a {len(hospit_non_chronique)} hospitalisations non chroniques.")
print(f"Cela concerne {nb_patients_non_chronique} patients.")
print("Tous les patients de la base ont au moins une hospitalisation qui n'est pas chronique.")
print(f"On a enlevé, en tout, {len(hospit_init) - len(hospit_non_chronique)} hospitalisations.")

Il y a 85594 hospitalisations non chroniques.
Cela concerne 10051 patients.
Tous les patients de la base ont au moins une hospitalisation qui n'est pas chronique.
On a enlevé, en tout, 140533 hospitalisations.


#### Profiling sur les hospitalisations après processing et pas chroniques

In [67]:
profile_hospit_non_chroniques = ProfileReport(hospit_non_chronique, title="Hospitalisations Non Chroniques")
profile_hospit_non_chroniques.to_file("profile_hospit_non_chr.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Export des bases de données nettoyées

In [77]:
hospit.to_csv("base_full_hospit_anonyme.csv", index=False)

In [78]:
hospit_non_chronique.to_csv("base_hospit_non_chr_anonyme.csv", index=False)

In [79]:
patient.to_csv("patient_anonyme.csv", index=False)

In [80]:
consommant.to_csv("consommant_anonyme.csv", index=False)